In [1]:
import pandas as pd
import numpy as np
import os
import ta

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import subplots
from datetime import datetime,timedelta

In [2]:
#reading all csv from data folder and combine them all
def load_data():

    dfs = []

    for item in os.listdir('data'):
        df = pd.read_csv(f'data/{item}',
                        header=None,
                        names=['stock code','Date','Open','High','Low','Close','Volume','Netforeign'])
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df.dropna(inplace=True)
        df.set_index('Date',inplace=True)

        #sort values by date
        df.sort_values('Date',inplace=True)
        dfs.append(df)

    main_df = pd.concat(dfs)
    main_df.tail()
    
    ##################################################################################################
    #read tradeble stocks
    tradable = pd.read_csv('tradable.csv')
    
    #creating a new df of tradable stock
    tradable_stock_df = main_df[main_df['stock code'].isin(tradable['stock'])]
    tradable_stock_df.head()
    
    tradable_stock_list = tradable_stock_df['stock code'].unique()
    tradable_stock_list.sort()

    print(tradable_stock_list,len(tradable_stock_list))
    print("\n\n")
    
    #group by tradable stock
    tradable_stock_df=tradable_stock_df.groupby('stock code')
    
    return tradable_stock_df

In [3]:
def load_trade_list():
    trade_list = pd.read_csv('trade_list.csv')
    trade_list['Date'] = pd.to_datetime(trade_list['Date'])
    trade_list.dropna(inplace=True)
    trade_list.set_index('Date',inplace=True)
    #group by tradable stock
    trade_list=trade_list.groupby('Stock')
    
    return trade_list


In [4]:
def test_chart(data_copy,buy,sell):
    price = go.Candlestick(x=data_copy.index,
                   open=data_copy['Open'],
                   high=data_copy['High'],
                   low=data_copy['Low'],
                   close=data_copy['Close'],
                   name='prices')
    ##################################################################
    buys = go.Scatter(
            x = buy.index,
            y = buy['Price'],
            mode = 'markers',
            marker = {'color': 'skyblue'},
            name = 'buy'

        )

    sells = go.Scatter(
            x = sell.index,
            y = sell['Price'],
            mode = 'markers',
            marker = {'color': 'red'},
            name = 'sell'

        )
    
    mafast = go.Scatter( 
        x = data.index,
        y = data['trend_sma_fast'],
        name = 'fast moving average')
    
    maslow = go.Scatter( 
        x = data.index,
        y = data['trend_sma_slow'],
        name = 'slow moving average')

    ##################################################################

    volume = go.Bar( 
            x = data_copy.index,
            y = data_copy['Volume'],
            #fill= 'tozeroy',
            marker = {'color': 'green'},
            name= 'volume')

    layout = go.Layout(
        height = 700,
        xaxis = dict(
            rangeslider = dict(
                visible = False
            ),
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                        label='YTD',
                        step='year',
                        stepmode='todate'),
                    dict(count=1,
                        label='1y',
                        step='year',
                        stepmode='backward'),
                    dict(step='all')
                ])
            ),
        ),
        yaxis = dict(
            autorange=True
            )
        )

    ######################################################################################

    ######################################################################################
    #data = [price,ma20,ma50,ma100]

    #data = [trace1,trace2]
    #fig = go.Figure(data=prices, layout=layout)
    fig = subplots.make_subplots(rows=4, cols=1,specs=[[{'rowspan': 3}],
                                                    [None],
                                                    [None],
                                                    [{}]],
                              shared_xaxes=True
                             )


    fig.append_trace(price, 1,1)
    fig.append_trace(buys, 1,1)
    fig.append_trace(sells, 1,1)
#     fig.append_trace(mafast, 1,1)
#     fig.append_trace(maslow, 1,1)

    fig.append_trace(volume, 4,1)
    #fig = dict(data=data, layout=layout)

    fig['layout'].update(layout)


    iplot({
        "data": fig
    })

In [5]:
stock_data = load_data()
trade_data = load_trade_list()

['ABA' 'ABG' 'AC' 'ACEN' 'ACEX' 'AEV' 'AGI' 'ALCO' 'ALI' 'ALLHC' 'ANI'
 'AP' 'APC' 'APX' 'AXLM' 'BCOR' 'BDO' 'BLOOM' 'BPI' 'BRN' 'C' 'CEB' 'CHP'
 'CIC' 'CNPF' 'COSCO' 'CPG' 'CPM' 'DD' 'DITO' 'DMC' 'DNL' 'EAGLE' 'EEI'
 'EMP' 'EW' 'FB' 'FGEN' 'FLI' 'FNI' 'FOOD' 'FRUIT' 'GERI' 'GLO' 'GTCAP'
 'HLCM' 'HOME' 'HOUSE' 'ICT' 'IDC' 'IMI' 'ION' 'IRC' 'JFC' 'JGS' 'KPPI'
 'LPZ' 'LR' 'LTG' 'MAC' 'MAH' 'MAXS' 'MBT' 'MEG' 'MER' 'MHC' 'MPI' 'MRC'
 'MRSGI' 'MWC' 'MWIDE' 'NIKL' 'NOW' 'ORE' 'PCOR' 'PGOLD' 'PHA' 'PHES'
 'PIP' 'PIZZA' 'PLC' 'PNB' 'PNX' 'PRMX' 'PX' 'PXP' 'RCB' 'RLC' 'RRHI'
 'RWM' 'SCC' 'SECB' 'SHLPH' 'SLI' 'SM' 'SMC' 'SMPH' 'SSI' 'STI' 'STR'
 'TBGI' 'TECH' 'TEL' 'TUGS' 'UBP' 'URC' 'VITA' 'VLL' 'VUL' 'WIN' 'WLCON'
 'WPI' 'X' '^PSEi'] 114





In [12]:
stock = "JFC"

In [13]:
#hide warnings
import warnings
warnings.filterwarnings('ignore')

data = stock_data.get_group(stock)
data.sort_values('Date',inplace=True)

#adding technical indicators
data = ta.add_all_ta_features(data,
                          open="Open",
                          high="High",
                          low="Low",
                          close="Close",
                          volume="Volume",
                          fillna=False)

# print(data.columns)

t_data = trade_data.get_group(stock)
t_data.sort_values('Date',inplace=True)

buy_sell = t_data.groupby('Action')

buy = buy_sell.get_group(1)
buy.sort_values('Date',inplace=True)

sell = buy_sell.get_group(2)
sell.sort_values('Date',inplace=True)


# print(len(data),len(t_data))
# print(data.tail())
# print(t_data.tail())
# print('buy\n',buy.tail())
# print('sell\n',sell.tail())
#hide warnings
import warnings
warnings.filterwarnings('ignore')



In [14]:
test_chart(data,buy,sell)